In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X,y=make_classification(n_samples=1000, n_features=10, n_informative=2,n_redundant=8,
                        weights=[0.9,0.1], flip_y=0, random_state=42)
np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, stratify=y , random_state=42)

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
smt=SMOTETomek(random_state=42)

In [ ]:
X_train_res, y_train_res=smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [ ]:
models=[
    (
        "Logistic Regression",
        {"C":1, "solver":"liblinear"},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators":30, "max_depth":3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )

]

In [ ]:
reports=[]

In [ ]:
for model_name, params, model, train_set, test_set in models:
  X_train = train_set[0]
  y_train = train_set[1]
  X_test = test_set[0]
  y_test = test_set[1]

  model.set_params(**params)
  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  report=classification_report(y_test, y_pred, output_dict=True)
  reports.append(report)

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [ ]:
import mlflow

mlflow.set_experiment("Anomaly Detection")
# Optional: set your tracking URI if remote
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name) as run:
        run_id = run.info.run_id

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        # Log model
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

        # ✅ Register the model (inside the active run)
        model_uri = f"runs:/{run_id}/model"
        mlflow.register_model(model_uri=model_uri, name=model_name)

        print(f"✅ Model {model_name} registered successfully with run_id: {run_id}")


2025/09/11 04:12:37 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2025/09/11 04:12:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/11 04:12:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Logistic Regression'.
2025/09/11 04:12:48 WARNING mlflow.tracking._model_registry.fluent: Run with id ebca1e63d55c40d5a9a491115c4c4374 has no artifacts at artifact path 'model', registering model based on models:/m-6125aa3e494c4a19b1edc5d1c2f38f49 instead
Created version '1' of model 'Logistic Regression'.
2025/09/11 04:12:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model Logistic Regression registered successfully with run_id: ebca1e63d55c40d5a9a491115c4c4374


2025/09/11 04:12:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Random Forest'.
2025/09/11 04:12:54 WARNING mlflow.tracking._model_registry.fluent: Run with id 1af3a8586aec409b81d121514dc15301 has no artifacts at artifact path 'model', registering model based on models:/m-c52b5b5962d94e8390781f30921d2438 instead
Created version '1' of model 'Random Forest'.
2025/09/11 04:12:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model Random Forest registered successfully with run_id: 1af3a8586aec409b81d121514dc15301


2025/09/11 04:12:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBClassifier'.
2025/09/11 04:12:58 WARNING mlflow.tracking._model_registry.fluent: Run with id 707b75949e6547b6b7f63f6f5e9ad6af has no artifacts at artifact path 'model', registering model based on models:/m-439d7f1f72a74047b7c1a6bb59b0f0e4 instead
Created version '1' of model 'XGBClassifier'.
2025/09/11 04:12:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model XGBClassifier registered successfully with run_id: 707b75949e6547b6b7f63f6f5e9ad6af


2025/09/11 04:13:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBClassifier With SMOTE'.
2025/09/11 04:13:01 WARNING mlflow.tracking._model_registry.fluent: Run with id 19272b84f62f4a299c0dfc04114387c8 has no artifacts at artifact path 'model', registering model based on models:/m-4d84789eb56c49569168fa7469080326 instead


✅ Model XGBClassifier With SMOTE registered successfully with run_id: 19272b84f62f4a299c0dfc04114387c8


Created version '1' of model 'XGBClassifier With SMOTE'.


In [ ]:
mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="XGB-Smote")


Successfully registered model 'XGB-Smote'.
2025/09/11 04:13:01 WARNING mlflow.tracking._model_registry.fluent: Run with id 19272b84f62f4a299c0dfc04114387c8 has no artifacts at artifact path 'model', registering model based on models:/m-4d84789eb56c49569168fa7469080326 instead
Created version '1' of model 'XGB-Smote'.


<ModelVersion: aliases=[], creation_timestamp=1757563981677, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1757563981677, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-4d84789eb56c49569168fa7469080326', run_id='19272b84f62f4a299c0dfc04114387c8', step=0, timestamp=1757563978345, value=0.9633333333333334>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score_macro', model_id='m-4d84789eb56c49569168fa7469080326', run_id='19272b84f62f4a299c0dfc04114387c8', step=0, timestamp=1757563978345, value=0.8996319839411174>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_0', model_id='m-4d84789eb56c49569168fa7469080326', run_id='19272b84f62f4a299c0dfc04114387c8', step=0, timestamp=1757563978345, value=0.9777777777777777>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_1', model_id='m-4d84789eb56c49569168fa7469080326', run_id='19272b84f62f4a299c0dfc04114387c8', step

In [ ]:
import mlflow.xgboost

model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)

# Predict
y_pred = loaded_model.predict(X_test)
print(y_pred[:])


[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1
 1 0 0 0]


In [ ]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [ ]:
from mlflow.tracking import MlflowClient

model_name = "XGB-Smote"
production_model_name = "anomaly-detection-prod"
source_version = 1

client = MlflowClient()

# (Optional) set alias for clarity
client.set_registered_model_alias(model_name, "challenger", version=source_version)

# Make sure destination exists
try:
    client.create_registered_model(production_model_name)
except Exception:
    pass  # Already exists


# Copy the version
client.copy_model_version(
    src_model_uri=f"models:/{model_name}@challenger",
    dst_name=production_model_name
)

#client.set_registered_model_alias(production_model_name, "champion", new_version.version)

Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1757563981754, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1757563981754, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='19272b84f62f4a299c0dfc04114387c8', run_link=None, source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
model_version = 1
prod_model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

# Names and versions
model_name = "XGB-Smote"                 # source registered model
production_model_name = "anomaly-detection-prod"  # target registered model
source_version = 1                       # source version to copy

client = MlflowClient()

# (Optional) set alias on the source model for clarity
client.set_registered_model_alias(model_name, "challenger", version=source_version)

# Make sure destination registered model exists
try:
    client.create_registered_model(production_model_name)
except Exception:
    pass  # already exists

# Copy challenger version into production registry
new_version = client.copy_model_version(
    src_model_uri=f"models:/{model_name}@challenger",
    dst_name=production_model_name
)

# Assign alias 'champion' to the copied version
client.set_registered_model_alias(
    production_model_name, "champion", new_version.version
)

# Load the champion model
prod_model_uri = f"models:/{production_model_name}@champion"
loaded_model = mlflow.xgboost.load_model(prod_model_uri)

# Run predictions
y_pred = loaded_model.predict(X_test)
print(y_pred[:4])


[0 0 0 0]


Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '2' of model 'anomaly-detection-prod'.


In [ ]:
from fastapi import FastAPI
import mlflow.pyfunc

In [ ]:
app=FastAPI()

In [ ]:
model=mlflow.pyfunc.load_model("models:/anomaly-detection-prod@champion")

In [ ]:
@app.get("/")
def root():
  return{"message": "MLflow model is live!"}

In [ ]:
@app.post("/predict")
def predict(data: dict):
  X=[list(data.values())]
  y_pred=model.predict(X)
  return {"prediction": y_pred.tolist()}

In [ ]:
#mlfow wraps the ml model, after wrapping it predicts output in json format

In [ ]:
%%writefile serve.py
from fastapi import FastAPI
import mlflow
import pandas as pd
app=FastAPI()
model=uri="models:/anomaly-detection-prod@champion"
model=mlflow.pyfunc.load_model(model_uri)

@app.get("/")
def home():
  return{"message": "mlflow model is live inside Docker!"}

@app.post("/predict")
def predict(data: dict):
  """
  Expects input like:
  {
    "feature1":0.5,
    "feature2":1.2,
    "feature3":-0.8

  }
  """
  df=pd.DataFrame([data])
  prediction=model.predict(df)
  return {"prediction": prediction.tolist()}


Writing serve.py


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile Dockerfile
#Base image with python
FROM python:3.9

#set working directory inside container
WORKDIR /app

#copy files
COPY serve.py /app
COPY requirements.txt /app

#Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

#expose FASTAPI port
EXPOSE 8000

#start fastapi app
CMD ["uvicorn", "serve:app", "--host", "0.0.0.0", "--port", "8000"]


Overwriting Dockerfile


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#docker engine code
"""
{
  "builder": {
    "gc": {
      "defaultKeepStorage": "20GB",
      "enabled": true
    }
  },
  "experimental": false,
  "registry-mirrors": [
    "https://mirror.gcr.io"
  ]
}

then pull ur python version
docker pull python:3.10-slim
"""

'\n{\n  "builder": {\n    "gc": {\n      "defaultKeepStorage": "20GB",\n      "enabled": true\n    }\n  },\n  "experimental": false,\n  "registry-mirrors": [\n    "https://mirror.gcr.io"\n  ]\n}\n\nthen pull ur python version \ndocker pull python:3.10-slim\n'